In [1]:
import pickle

# KBO 클래스 선언
class KBOTeam:
    def __init__(self):
        self.win_lose = {}
        self.corpus = {}
    
    def addWinLose(self, date, value):    # 승패 정보 딕셔너리에 데이터 추가
        self.win_lose[date] = value
    
    def addCorpus(self, date, corpus):    # 코퍼스 데이터 딕셔너리에 데이터 추가
        self.corpus[date] = corpus
    
    def __str__(self):    # 객체 자체를 출력했을 때, 출력 형식을 정함
        return f'{self.win_lose}, {self.corpus}'

In [2]:
# 파일 불러오기

# 구단 이름 명 = 피클 파일명
kbo_team_names = ['samsung', 'kia', 'lotte']
kbo_teams = []    # 구단 객체를 담을 배열
for name in kbo_team_names:
    with open(f'../../datasets/kbo_corpus/{name}.pickle', 'rb') as fr:
        kbo_teams.append(pickle.load(fr))

In [3]:
# 파일 입출력 함수 선언

# 파일 Write
def write_text(doc_arr, filename):    # win/lose_filename 형태
    # string형태로 합치기
    samsung_str = '\n'.join(doc_arr[0])
    kia_str = '\n'.join(doc_arr[1])
    lotte_str = '\n'.join(doc_arr[2])
    # 파일로 쓰기
    with open(f"../../datasets/kbo_corpus/samsung_{filename}.txt", 'w') as fw:
        fw.write(samsung_str)
    with open(f"../../datasets/kbo_corpus/kia_{filename}.txt", 'w') as fw:
        fw.write(kia_str)
    with open(f"../../datasets/kbo_corpus/lotte_{filename}.txt", 'w') as fw:
        fw.write(lotte_str)

# 파일 Read
def read_text(filename):
    results = []
    with open(f"../../datasets/kbo_corpus/samsung_{filename}.txt", 'r') as fr:
        samsung = fr.read()
    results.append(samsung.split('\n'))
    with open(f"../../datasets/kbo_corpus/kia_{filename}.txt", 'r') as fr:
        kia = fr.read()
    results.append(kia.split('\n'))
    with open(f"../../datasets/kbo_corpus/lotte_{filename}.txt", 'r') as fr:
        lotte = fr.read()
    results.append(lotte.split('\n'))
    return results

In [4]:
# 팀별 코퍼스 데이터만 분리하여 저장

teams_corpus = []

for team in kbo_teams:
    corpus = []
    for date in team.corpus:
        corpus += team.corpus[date]
    teams_corpus.append(corpus)

In [5]:
for i in range(3):
    print(len(teams_corpus[i]))
write_text(teams_corpus, 'document')

20302
13850
22421


In [6]:
# 전처리 시작
# <1> Basic Processing
# 1. 문장 부호 대체
# 2. 문장 토큰화는 생략
def clean_punc(text):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    
    return text.strip()    # 공백 제거

In [7]:
documents = read_text('document')

punc1_cleaned_documents = [[], [], []]

for i in range(3):
    for sen in documents[i]:
        punc1_cleaned_documents[i].append(clean_punc(sen))

punc1_cleaned_documents[1][:5]

['정해영 올림픽 간다고?한심 사토한테 털릴준비해라',
 '작년에 황대인 기회 안준거 진짜 아깝다',
 '작년 머인이 1군무대 처음으로 100타석이상 한건데',
 '이민우 17데뷔승 이후 첫승이냐',
 '야구장 갈라면 케텍스에서 한참 가야되잖아 니네']

In [8]:
# 3. 문장 부호, 숫자, html 태그, 공백 제거 + 대문자 소문자로
import re

def clean_corpus(origin_texts):
    cleaning_texts = []
    for sen in origin_texts:    # r'문자열' : raw 문자열. \기호 그대로 출력
        cleaning_sen = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '', sen)    #remove punctuation
        # cleaning_sen = re.sub(r'\d+', '', cleaning_sen)    # remove number
        cleaning_sen = cleaning_sen.lower()    # lower case
        cleaning_sen = re.sub(r'\s+', ' ', cleaning_sen)    # remove extra space
        cleaning_sen = re.sub(r'<[^>]+>', '', cleaning_sen)    # remove spaces
        cleaning_sen = re.sub(r'^\s+', '', cleaning_sen)    # remove space from start
        cleaning_sen = re.sub(r'\s+$', '', cleaning_sen)    # remove space from the end
        cleaning_texts.append(cleaning_sen)
    return cleaning_texts

In [9]:
punc_cleaned_documents = []

for i in range(3):
    punc_cleaned_documents.append(clean_corpus(punc1_cleaned_documents[i]))

punc_cleaned_documents[1][:5]

['정해영 올림픽 간다고한심 사토한테 털릴준비해라',
 '작년에 황대인 기회 안준거 진짜 아깝다',
 '작년 머인이 1군무대 처음으로 100타석이상 한건데',
 '이민우 17데뷔승 이후 첫승이냐',
 '야구장 갈라면 케텍스에서 한참 가야되잖아 니네']

In [10]:
# <2> Spell check --> 생략
# <3> 단어 토큰화 + Stemming

# 1. 단어 토큰화 : khaiii 라이브러리 사용
from khaiii import KhaiiiApi

# 명사, 형용사, 동사, 부사만 남겨 둠
def pos_corpus(document):
    api = KhaiiiApi()
    significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']
    
    corpus = []
    for sen in document:
        if sen == '':    # 예외처리
            continue
        pos_tagged = ''
        for word in api.analyze(sen):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += str(morph) + ' '
        corpus.append(pos_tagged.strip())
        # 비교
        # print(sen)
        # print(pos_tagged.strip())
    return corpus

In [12]:
pos_tagged_documents = []

for i in range(3):
    pos_tagged_documents.append(pos_corpus(punc_cleaned_documents[i]))
pos_tagged_documents[1][:5]

['정해영/NNP 올림픽/NNG 가/VV 한심/NNG 사토/NNG 털릴준비/NNG 하/XSV',
 '작년/NNG 황대/NNG 기회/NNG 안준거/NNP 진짜/MAG 아깝/VA',
 '작년/NNG 머인/NNG 군무대/NNG 처음/NNG 타석이상/NNG 한/NNG 것/NNB',
 '이민우/NNP 데뷔승/NNG 이후/NNG 승/NNG',
 '야구장/NNG 가/VV 라/VV 케텍스/NNP 한참/NNG 가/VV 야/NNG 되/XSV 않/VX']

In [13]:
# 2. stemming 수행
def stemming_corpus(document):
    p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
    p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
    p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
    p4 = re.compile('[가-힣A-Za-z0-9]+/VX')
    
    corpus = []
    for sen in document:
        ori_sen = sen
        
        # 1번 경우
        mached_terms = re.findall(p1, ori_sen)  
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sen = ori_sen.replace(ori_terms, modi_terms)
        
        # 2번 경우
        mached_terms = re.findall(p2, ori_sen)  
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sen = ori_sen.replace(ori_terms, modi_terms)
        
        # 3번 경우
        mached_terms = re.findall(p3, ori_sen)  
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if modi_terms[-1] != '다':
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sen = ori_sen.replace(ori_terms, modi_terms)
            
        # 4번 경우
        mached_terms = re.findall(p4, ori_sen)  
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if modi_terms[-1] != '다':
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sen = ori_sen.replace(ori_terms, modi_terms)
        
        corpus.append(ori_sen)
    return corpus

In [14]:
stemming_documents = []
for i in range(3):
    stemming_documents.append(stemming_corpus(pos_tagged_documents[i]))
stemming_documents[1][:5]

['정해영/NNP 올림픽/NNG 가다/VV 한심/NNG 사토/NNG 털릴준비하다/VV',
 '작년/NNG 황대/NNG 기회/NNG 안준거/NNP 진짜/MAG 아깝/VA',
 '작년/NNG 머인/NNG 군무대/NNG 처음/NNG 타석이상/NNG 한/NNG 것/NNB',
 '이민우/NNP 데뷔승/NNG 이후/NNG 승/NNG',
 '야구장/NNG 가다/VV 라다/VV 케텍스/NNP 한참/NNG 가다/VV 야되다/VV 않다/VV']

In [15]:
write_text(stemming_documents, 'cleaned')